# Reset_ESKM_eventlog

<div class="alert alert-block alert-info" style="border-radius: 20px;">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>resetting the ESKM event log (ESKM stand for Enterprise Secure Key Manager)</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success" style="border-radius: 20px;">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li><a href = "https://www.json.org/json-en.html">json </a></li>
    <li><a href = "https://pypi.org/project/redfish/">redfish</a></li>
    <li><a herf= "##">RedfishClient</a></li>
    <li><a href= "#">ServerDownOrUnreachableError</a></li>
    
</div>

In [6]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

ImportError: cannot import name 'RedfishClient' from 'redfish' (/home/tanveer/.local/lib/python3.10/site-packages/redfish/__init__.py)

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h1> 2. </h1>Below is the overview of reset_ESKM_eventlog function. In this function we are providing secure management of cryptographic keys for various purposes for example encryption and authentication.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present for secure key manager.

1. In the <b></b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using<b> get_resource_directory module.</b>

    <br>

2. If the resource directory is enabled or we want to force it's non-use then navigate to the directory where managers members data is present. 
   Utilising the <b>HPE </b>(Hewlett Packard Enterprise) OEM Redfish implementation to retrieve the URI for the ESKM (Enterprise Security Key Manager) service.
   By making a Redfish API call to reset the event log of the HPE Enterprise Secure Key Manager (ESKM) security service of an HPE server. It first tries to obtain the URI for the ESKM service using the HPE-specific resource directory, but if that is not available or disabled, it falls back to navigating the Redfish API object model to find the necessary URIs for the relevant managers and security service.
   Once the URI for the ESKM service is obtained, a POST request is sent to the <b>ClearESKMLog</b> action using the <b>post()</b> method of the Redfish API object, with an empty dictionary in the body of the request.
   
    <br>

3. Else It iterates through a list of resource instances and checks if the instance type contains <b>#HpeESKM</b> in it. If found, it sets the URI of that resource instance as the ESKM URI and breaks out of the loop. 
    
    
<br>
    
4. If above function retrive the URI for the ESKM service from the HPE-specific <b>Links</b> object then  we retrieves the URI for a specific action to clear the ESKM log on an HPE server's Security Service ESKM resource, using the Redfish API. It then sets the action to be executed on the server in the request body.
    
    <br>
    
    If the response of this POST request was 400 it means there was some error from server-side while doing the same operation and we inform the user about the error by It attempts to extract and pretty-print the extended error message information, but if this fails, it writes an error message to the standard error stream indicating that it was unable to access the iLO Extended Message Info.
    <br>
    Else there can also be an error which we aren't aware of, in that case we print out the response status code alongwith erro     message .<br>
    Ultimately, if everything went properly we print out the response object along with success message.
    

 </div>

In [3]:
def reset_ESKM_eventlog(_redfishobj):

    body = dict()
    security_service_eskm_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links\
                                                                ['SecurityService']['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        security_service_eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeESKM.' in instance['@odata.type']:
                security_service_eskm_uri = instance['@odata.id']
                break

    if security_service_eskm_uri:
        security_service_eskm_uri = _redfishobj.get(security_service_uri).obj.Links\
                                                                            ['ESKM']['@odata.id']
        security_service_eskm_resp = _redfishobj.get(security_service_eskm_uri)
        security_service_eskm_clearlog_uri = security_service_eskm_resp.obj['Actions']\
                                                                ['#HpeESKM.ClearESKMLog']['target']
        body["Action"] = "HpeESKM.ClearESKMLog"
        #We pass the URI and the dictionary as a POST command (part of the redfish object)
        resp = _redfishobj.post(security_service_eskm_clearlog_uri, body)
        #If iLO responds with something outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong

        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h1> 3. </h1>This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> object is being created mainly to log into the system with valid credentials.
     
    <br>

3.  We try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server  is not avaialbe we get an error saying "server unreachable"<br>
    <br>
4. Then we call <b>reset_ESKM_eventlog</b>function with our redfish object and the function does the reset the ESKM_eventlog.
    
    <br>
    

5. Lastly, REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

In [1]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    reset_ESKM_eventlog(REDFISHOBJ)
    REDFISHOBJ.logout()

NameError: name 'ServerDownOrUnreachableError' is not defined

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>